In [2]:
# import general packages:
import numpy as np
import pandas as pd
from pandas import Timestamp
import os
import datetime as dt
import time 
import itertools
from math import sqrt
import csv
from datetime import datetime
# import timestring
from IPython.core.display import display, HTML
from IPython.display import Image
display(HTML("<style>.container { width:100% !important; }</style>"))

# import visualization packages:
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly import tools

In [3]:
import re

In [8]:
import tkinter as tk
from tkinter import filedialog

In [4]:
# standardize dataframe colume names
def col_name(df):
    """
    this is to trim the data_frame column names to a unique format:
    all case, replace space to underscore, remove parentheses
    param df:
        raw from share drive for
    return:
        polished data set with new column names
    """
    df.columns = df.columns.str.strip().str.lower().str.replace('-','').str.replace(' ', '_').str.replace('(', '').\
                    str.replace(')', '').str.replace('"','')
    return df


def preprocess_fill_na(df):
    df_all = df.copy()
    df_price = df_all[list(df_all.loc[:, '1_piece_bucket': 'standard_cost'])]
    df_price = df_all.loc[:, '1_piece_bucket': 'standard_cost'].replace({0:np.nan})
    for index, row in df_price.iterrows():
        if row.isnull().all():
            df_all.loc[index, '1_piece_bucket': '101_piece_bucket'] = df_all.loc[index, 'standard_cost']
        elif not np.isnan(row[0]) :
            df_all.loc[index, '1_piece_bucket': '101_piece_bucket'] = row.ffill()
        else:
            valid_col_index = row.first_valid_index()
            df_all.loc[index, '1_piece_bucket': valid_col_index] = row[valid_col_index]
            df_all.loc[index, valid_col_index:] = row[valid_col_index:].ffill()
    return df_all


def get_vop_qty(pricing_row, vop_day = 1):
    """
    pricing_row: data series, single entire row from df_result
    vop_days: iterating from days selection
    """
    vop_qty = (pricing_row['eau']/320)*vop_day
    return int(vop_qty) + 1

def get_order_qty(pricing_row, vop_qty):
    """
    pricing_row: data series, single entire row from df_result
    vop_qty: iterated qty from VOP days calculation
    """
    mlpq = pricing_row['multiple_order_qty'] 
    minq = pricing_row['minimum_reorder_qty']

    if (mlpq == 0) & (minq == 0):
        vopq = vop_qty
    
    elif (mlpq == 0) & (minq != 0):
        vopq = max(vop_qty, minq)
    
    elif (minq == 0) & (mlpq != 0):
        if vop_qty <= mlpq:
            vopq = mlpq
        else:
            vopq = ((vop_qty // mlpq)+1) * mlpq
            
    else:
        if mlpq <= minq: 
            if vop_qty <= minq:
                vopq = ((minq // mlpq)+1) * mlpq
            else:
                vopq = ((vop_qty // mlpq)+1) * mlpq    
                
        else:
            if vop_qty < mlpq:
                vopq = mlpq
            else:
                vopq = ((vop_qty // mlpq)+1) * mlpq                
    return vopq 


def get_unit_cost(pricing_row, pricing_break, order_qty):
    """
    pricing_row: data series, single entire row from df_result
    pricing_break: dictionary: pricing category with breaking range
    order_qrt: is vopq after comapring with MOQ & MINQ iterated qty from VOP days calculation
    
    return: unit_cost: pricing from break bucket
    """
    for key, value in pricing_break.items():
        if (value[0] <= order_qty) and (order_qty < value[1]):
            unit_cost = pricing_row[key]            
            return key, unit_cost           
    return None


def get_purchasing_cost(unit_cost, pricing_row):
    """
    unit_cost: int, result from pricing break
    pricing_row: data series, single entire row from df_result
    return: total purchasing cost per item
    """
    purchasing_cost = unit_cost * pricing_row['eau']
    return round(purchasing_cost,2)


def get_order_frequency(order_qty, pricing_row):
    """
    order_qty: ordering qty result from function get_order_qty
    pricing_row: data series, single entire row from df_result
    return: 
    """
    order_frequency = min(pricing_row['eau']/order_qty, 52)
    return round(order_frequency,2)


def get_holding_cost(purchasing_cost, order_frequency, financial_rate=0.125):
    '''
    Order Frequency clarification: 
        annual expenditure * financial opportunity capital = annual captical cost, divided by order frequency gets each order cycle's capital cost
        for example:  annual purchasing cost = $10000, if vop = 83 >> order frequency = 4 times, then annualized capital cost = 10000 * 0.1 / 4 = 250
                                                       if vop = 7 >> order frequency = 50 times, then annualized capital cost = 10000 * 0.1 / 50 = 20                                                       
    '''
    financial_carrying_frequency = max(order_frequency, 365/361)
    holding_cost = (purchasing_cost * financial_rate)/financial_carrying_frequency 
    return round(holding_cost,2)


def get_logistic_cost(order_frequency):
    logistic_cost = order_frequency*99/5
    return round(logistic_cost,2)

# def get_holding_cost(purchasing_cost, vop_day, financial_rate=0.125):
#     m = vop_day/365
#     holding_cost = purchasing_cost * financial_rate * m
#     return round(holding_cost,2)

In [5]:
def get_optimal_vops(pricing_row, cost_plot = False, summary = True, financial_rate = 0.073):
    
    cost_dict = {'purchasing_cost': [],
                'holding_cost' : [],
                'logistic_cost' : [],
                'total_cost' : []}  
    
    unit_cost_list = []
    
    min_combined_cost = np.inf
    vop_day = 1
    optimal_unit_cost = 0
    min_purchasing_cost = 0
    min_holding_cost = 0
    min_logistic_cost = 0
    min_combined_cost = np.inf
    optimal_vop_day = 0
    optimal_vop_qty = 0
    optimal_vop_freq = 0
    optimal_pricing_bucket = 0
    
#     while vop_day < 84:
    for vop_day in range (1, 84): # due to maximum order freqency is 52, hence vop_day starts on 7 days

        vop_qty = get_vop_qty(pricing_row=pricing_row, vop_day = vop_day)
        order_qty = get_order_qty(pricing_row=pricing_row, vop_qty = vop_qty)
        pricing_bucket, unit_cost = get_unit_cost(pricing_row, pricing_break, order_qty=order_qty)
        purchasing_cost = get_purchasing_cost(unit_cost = unit_cost, pricing_row= pricing_row)
        order_frequency = get_order_frequency(order_qty, pricing_row)
        holding_cost = get_holding_cost(purchasing_cost, order_frequency=order_frequency, financial_rate=financial_rate)
        logistic_cost = get_logistic_cost(order_frequency=order_frequency)
        combined_cost = round((purchasing_cost + holding_cost + logistic_cost), 2)
        if combined_cost < min_combined_cost:
            min_purchasing_cost = purchasing_cost
            min_holding_cost = holding_cost
            min_logistic_cost = logistic_cost
            min_combined_cost = combined_cost
            optimal_vop_day = vop_day
            optimal_vop_qty = order_qty
            optimal_unit_cost = unit_cost
            optimal_vop_freq = order_frequency
            optimal_pricing_bucket = pricing_bucket
        
        unit_cost_list.append(unit_cost)

        cost_dict['purchasing_cost'].append(purchasing_cost)
        cost_dict['holding_cost'].append(holding_cost)
        cost_dict['logistic_cost'].append(logistic_cost)
        cost_dict['total_cost'].append(combined_cost)
        
#         vop_day += 1
    
    cost_df = pd.DataFrame(cost_dict)
        
    if cost_plot:
        x_vops = list(np.arange(1,84))
        fig = make_subplots(rows=4, cols=1)
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['holding_cost'], mode = 'lines', name = 'holding_cost'))
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['logistic_cost'], mode = 'lines', name = 'logistic_cost'))
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['purchasing_cost'], mode = 'lines', name = 'purchasing_cost'), row=2, col=1)
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['total_cost'], mode = 'lines', name = 'total_cost'), row=3, col=1)
        fig.add_trace(go.Scatter(x = x_vops, y = unit_cost_list, mode = 'lines', name = 'unit_cost'), row=4, col=1)
        fig.show()
        
    if summary:
        print(f'Material: {pricing_row.part_number}, {pricing_row.part_description}')
        print(f'vop_day: {optimal_vop_day} (days cycle)')
        print(f'EAU: {pricing_row.eau} (ea/year)\nEach order qty:{optimal_vop_qty} (ea/order)\norder frequency: {optimal_vop_freq} (times/year)')
        print(f'pricing bucket: {optimal_pricing_bucket}\nunit cost: {optimal_unit_cost}')
        print(f'material purchasing cost: {min_purchasing_cost} (dollars/year)\nholding cost:{min_holding_cost} (dollars/year)\nlogistic cost:{min_logistic_cost} (dollars/year)')
        print(f'total_landing_cost_optimal: {min_combined_cost} (dollars/year)\n')

    return optimal_vop_day, optimal_vop_qty, optimal_vop_freq

## Start to run model 

In [59]:
# file selection
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [60]:
file_path

'S:/OSK-Share/DEPT/LOGISTICS/LC3/Projects/Logistics Optimization Tool/Files for Nate to Upload - VOP/VOP_Uploading/test_file_Nathan.xlsx'

In [61]:
pd.set_option('display.max_columns', None)
df_pricing = col_name(pd.read_excel(file_path))
df_result = preprocess_fill_na(df_pricing)

C:\Users\U279014\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

C:\Users\U279014\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [62]:
df_result

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost
0,3888774,"XMSN,4000RDS,P,TC521,PROG,1710",1,0,0,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.6700
1,3707003,"STEERING GEAR,XD120",5,0,0,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.2800
2,4110148,"MUF,DPF,ISX,HOR 5.0 S-S,2013",13,0,0,5265.00,5265.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5208.2493
3,4310295,"FAN DRV,2-SPD,8GRV,211,BRKT",74,0,0,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.7000
4,4020004,"MOTOR,PIST 2.1 F BI B KEYED",37,0,0,1313.90,1259.60,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.5000
5,4406838,"WIRE HARN,CAB,LOWER MAIN",36,0,0,1473.07,1348.64,1326.02,1326.02,1326.02,1192.27,1192.27,1192.27,1192.27,1192.27,1192.2700
6,1984700,"SPINDLE,FRONT AXLE",170,0,0,896.60,589.10,372.21,308.15,277.66,263.93,250.25,250.25,250.25,250.25,250.2500
7,15500FX,"SHAFT,AXLE 16TX14.500,BALL",183,0,0,1152.12,682.52,369.45,271.62,243.22,231.34,222.22,216.66,216.66,216.66,222.2200


In [63]:
sum(df_result.loc[:,'1_piece_bucket': 'standard_cost'].isna().sum())

0

In [64]:
df_result.columns

Index(['part_number', 'part_description', 'eau', 'minimum_reorder_qty',
       'multiple_order_qty', '1_piece_bucket', '2_piece_bucket',
       '6_piece_bucket', '16_piece_bucket', '31_piece_bucket',
       '51_piece_bucket', '101_piece_bucket', '251_piece_bucket',
       '501_piece_bucket', '1001_piece_bucket', 'standard_cost'],
      dtype='object')

In [65]:
pricing_break_keys = [c for c in df_result.columns if 'piece_bucket' in c]
pricing_break_keys

['1_piece_bucket',
 '2_piece_bucket',
 '6_piece_bucket',
 '16_piece_bucket',
 '31_piece_bucket',
 '51_piece_bucket',
 '101_piece_bucket',
 '251_piece_bucket',
 '501_piece_bucket',
 '1001_piece_bucket']

In [66]:
price_range = [int(re.search(r'\d+', c).group()) for c in pricing_break_keys]
price_range_2 = price_range[1:] + [np.inf]
pricing_break_values = list(zip(price_range,price_range_2))
pricing_break_values

[(1, 2),
 (2, 6),
 (6, 16),
 (16, 31),
 (31, 51),
 (51, 101),
 (101, 251),
 (251, 501),
 (501, 1001),
 (1001, inf)]

In [67]:
pricing_break = dict(list(zip(pricing_break_keys, pricing_break_values)))

In [68]:
pricing_break

{'1_piece_bucket': (1, 2),
 '2_piece_bucket': (2, 6),
 '6_piece_bucket': (6, 16),
 '16_piece_bucket': (16, 31),
 '31_piece_bucket': (31, 51),
 '51_piece_bucket': (51, 101),
 '101_piece_bucket': (101, 251),
 '251_piece_bucket': (251, 501),
 '501_piece_bucket': (501, 1001),
 '1001_piece_bucket': (1001, inf)}

In [69]:
df_result[df_result.loc[:,'1_piece_bucket': 'standard_cost'].isna().any(axis = 1)]

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost


In [70]:
# # quantity break for pricing break 
# pricing_break = {'1_piece_bucket': [1,2],
#  '2_piece_bucket': [2,5],
#  '5_piece_bucket': [5, 6],
#  '6_piece_bucket': [6, 16],
#  '16_piece_bucket': [16, 25],
#  '25_piece_bucket': [25, 31],
#  '31_piece_bucket': [31, 51],
#  '51_piece_bucket': [51, 101],
#  '101_piece_bucket': [101, np.inf]}

## Single item process >>>

In [51]:
random_pick = np.random.randint(-1, df_result.shape[0]-1)
single_item = df_result.copy().iloc[7,:]
single_item

part_number                               15500FX
part_description       SHAFT,AXLE 16TX14.500,BALL
eau                                           183
minimum_reorder_qty                             0
multiple_order_qty                              0
1_piece_bucket                            1152.12
2_piece_bucket                             682.52
6_piece_bucket                             369.45
16_piece_bucket                            271.62
31_piece_bucket                            243.22
51_piece_bucket                            231.34
101_piece_bucket                           222.22
251_piece_bucket                           216.66
501_piece_bucket                           216.66
1001_piece_bucket                          216.66
standard_cost                              222.22
Name: 7, dtype: object

### Optimal buy policy

In [52]:
fea_financial_rate = 0.1
a, b, c = get_optimal_vops(pricing_row=single_item,cost_plot=True, summary=True, financial_rate=fea_financial_rate)

Material: 15500FX, SHAFT,AXLE 16TX14.500,BALL
vop_day: 357 (days cycle)
EAU: 183 (ea/year)
Each order qty:205 (ea/order)
order frequency: 0.89 (times/year)
pricing bucket: 101_piece_bucket
unit cost: 222.22
material purchasing cost: 40666.26 (dollars/year)
holding cost:924.74 (dollars/year)
logistic cost:17.62 (dollars/year)
total_landing_cost_optimal: 41608.62 (dollars/year)



## Process in batch material file

In [71]:
vop_master_run = df_result.copy()

In [72]:
vop_master_run

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost
0,3888774,"XMSN,4000RDS,P,TC521,PROG,1710",1,0,0,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.67,12757.6700
1,3707003,"STEERING GEAR,XD120",5,0,0,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.28,1081.2800
2,4110148,"MUF,DPF,ISX,HOR 5.0 S-S,2013",13,0,0,5265.00,5265.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5147.00,5208.2493
3,4310295,"FAN DRV,2-SPD,8GRV,211,BRKT",74,0,0,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.70,768.7000
4,4020004,"MOTOR,PIST 2.1 F BI B KEYED",37,0,0,1313.90,1259.60,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.50,1208.5000
5,4406838,"WIRE HARN,CAB,LOWER MAIN",36,0,0,1473.07,1348.64,1326.02,1326.02,1326.02,1192.27,1192.27,1192.27,1192.27,1192.27,1192.2700
6,1984700,"SPINDLE,FRONT AXLE",170,0,0,896.60,589.10,372.21,308.15,277.66,263.93,250.25,250.25,250.25,250.25,250.2500
7,15500FX,"SHAFT,AXLE 16TX14.500,BALL",183,0,0,1152.12,682.52,369.45,271.62,243.22,231.34,222.22,216.66,216.66,216.66,222.2200


In [73]:
fea_financial_rate = 0.1

In [74]:
vop_master_run[['optimal_vop_day', 'optimal_vop_qty', 'optimal_vop_freq']] = vop_master_run.apply(lambda row: get_optimal_vops(row, financial_rate = fea_financial_rate) if row['eau'] != 0 else [0, 0, 0], axis=1, result_type='expand')

Material: 3888774, XMSN,4000RDS,P,TC521,PROG,1710
vop_day: 320 (days cycle)
EAU: 1 (ea/year)
Each order qty:2 (ea/order)
order frequency: 0.5 (times/year)
pricing bucket: 2_piece_bucket
unit cost: 12757.67
material purchasing cost: 12757.67 (dollars/year)
holding cost:1261.79 (dollars/year)
logistic cost:9.9 (dollars/year)
total_landing_cost_optimal: 14029.36 (dollars/year)

Material: 3707003, STEERING GEAR,XD120
vop_day: 83 (days cycle)
EAU: 5 (ea/year)
Each order qty:2 (ea/order)
order frequency: 2.5 (times/year)
pricing bucket: 2_piece_bucket
unit cost: 1081.28
material purchasing cost: 5406.4 (dollars/year)
holding cost:216.26 (dollars/year)
logistic cost:49.5 (dollars/year)
total_landing_cost_optimal: 5672.16 (dollars/year)

Material: 4110148, MUF,DPF,ISX,HOR 5.0 S-S,2013
vop_day: 124 (days cycle)
EAU: 13 (ea/year)
Each order qty:6 (ea/order)
order frequency: 2.17 (times/year)
pricing bucket: 6_piece_bucket
unit cost: 5147.0
material purchasing cost: 66911.0 (dollars/year)
holding

In [66]:
vop_master_run

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost,optimal_vop_day,optimal_vop_qty,optimal_vop_freq
0,1302130,NEVER SEEZ REGULAR GRADE LUBE,0,0,0,18.9500,18.9500,18.9500,18.9500,18.9500,18.9500,18.9500,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0
1,1304860,ELBOW 90 3.00ID RBR,3,0,0,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,12.3422,83.0,1.0,3.0


In [120]:
result_path = filedialog.askdirectory()
result_file_name = '(result)_'+os.path.split(file_path)[1]

In [121]:
result_path

'S:/OSK-Share/DEPT/LOGISTICS/LC3/Projects/Logistics Optimization Tool/Files for Nate to Upload - VOP/VOP_Result/FEA_result'

In [122]:
vop_master_run.to_excel(os.path.join(result_path, result_file_name), index=False)